In [ ]:
path_to_src = '/content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src'
dataset_name = "QM9"
dataset_metrics = ['r2','mae']
task_lst = ["mu","alpha","homo","lumo","gap","r2","zpve","u0","u298","h298","g298","cv"]
"""dataset_name = "PCBA"
task_lst = ["PCBA-1030","PCBA-1458","PCBA-1460","PCBA-2546","PCBA-2551","PCBA-485297","PCBA-485313","PCBA-485364","PCBA-504332","PCBA-504333","PCBA-504339","PCBA-504444","PCBA-504467","PCBA-588342","PCBA-624296","PCBA-624297","PCBA-624417","PCBA-651965","PCBA-652104","PCBA-686970","PCBA-686978","PCBA-686979","PCBA-720504"]
dataset_metrics = ['roc_auc','auprc']"""

#If using Google drive
from google.colab import drive
drive.mount('/content/drive')
# get the current working directory
import sys
sys.path.insert(0,path_to_src)

Mounted at /content/drive


In [ ]:
#!pip3 uninstall dgl -y
!pip3 install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html
!pip3 install dgllife
!pip3 install rdkit

Looking in links: https://data.dgl.ai/wheels/cu118/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 22.7 MB/s eta 0:00:00


# Calculate Similarity
### (only do once per dataset)

In [12]:
import torch
from motse import MoTSE

import pandas as pd
from utils.data import load_data, load_probe_data
from utils import load_model, makedir, set_random_seed
from trainer import Trainer
%load_ext autoreload
%autoreload 2
set_random_seed(22)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
#Train base GNNs on 10000 datapoints from each available source task

tasks = task_lst
data_path = path_to_src + f'/../datasets/{dataset_name}/10000/'
model_type = 'GCN'
model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/10000/"
makedir(model_path)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
results_dict = {'task':[]}

for task in tasks:
    print(task)
    train_loader, val_loader, test_loader, data_args = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[task],
        device = device
    )
    model = load_model(device=device)
    trainer = Trainer(device=device,tasks=[task],
                      data_args=data_args,model_path=model_path,
                     )
    model, task_results_dict = trainer.fit(model, train_loader,
                                      val_loader, test_loader)
    results_dict['task'].append(task)
    for metric in data_args['metrics']:
        if metric not in list(results_dict.keys()):
            results_dict.update({metric:[]})
        results_dict[metric].append(task_results_dict[metric][task])

In [14]:
#calculate similarity

model_type = 'GCN'
probe_data_path = path_to_src + '/../datasets/probe_data/zinc500/'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
motse = MoTSE(device)
probe_data = load_probe_data(probe_data_path)

source_tasks = target_tasks = task_lst
source_model_paths = []
for task in source_tasks:
    source_model_paths.append(path_to_src + f"/../saved_models/{dataset_name}/GCN/10000/{task}.pth")
target_model_paths = []
for task in target_tasks:
    target_model_paths.append(path_to_src + f"/../saved_models/{dataset_name}/GCN/1000/{task}.pth")
result_path = path_to_src + f"/../results/{dataset_name}/GCN/10000/zinc500/"

motse.cal_sim(source_tasks,target_tasks,
              source_model_paths,target_model_paths,result_path,probe_data)

500 loaded!
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/PCBA/GCN/10000/zinc500/.


# Transfer Knowledge

In [ ]:
import torch
import pandas as pd
from utils import load_model, makedir, set_random_seed
from utils.data import load_data
from trainer import Trainer

%load_ext autoreload
%autoreload 2
set_random_seed(22)#Prev run on 18, 20

task_lst = task_lst[:10] #Only testing transfer to first 10 tasks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#Pick best 3 source tasks for each target task based on MSE similarity

data_path = path_to_src + f'/../datasets/{dataset_name}/1000/'
model_type = 'GCN'
source_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/10000/"
model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/10000->1000/"
makedir(model_path)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
results_dict = dict()

source_target_pairs = []
similarity_results_path = path_to_src + f"/../results/{dataset_name}/GCN/10000/zinc500/"
for target_task in task_lst:
    similarity_df = pd.read_csv(similarity_results_path + target_task + '.csv')
    similarity_df = similarity_df.sort_values(by='motse', ascending=False)
    for i in range(1,4):
      source_task = similarity_df.iloc[[i]]['source_task'].to_list()[0]
      source_target_pairs.append((source_task, target_task))
print(source_target_pairs)

[('PCBA-485364', 'PCBA-1030'), ('PCBA-504332', 'PCBA-1030'), ('PCBA-1460', 'PCBA-1030'), ('PCBA-485297', 'PCBA-1458'), ('PCBA-485313', 'PCBA-1458'), ('PCBA-2546', 'PCBA-1458'), ('PCBA-504332', 'PCBA-1460'), ('PCBA-504333', 'PCBA-1460'), ('PCBA-2546', 'PCBA-1460'), ('PCBA-2551', 'PCBA-2546'), ('PCBA-1458', 'PCBA-2546'), ('PCBA-1460', 'PCBA-2546'), ('PCBA-2546', 'PCBA-2551'), ('PCBA-504333', 'PCBA-2551'), ('PCBA-1460', 'PCBA-2551'), ('PCBA-485313', 'PCBA-485297'), ('PCBA-1458', 'PCBA-485297'), ('PCBA-2546', 'PCBA-485297'), ('PCBA-485297', 'PCBA-485313'), ('PCBA-1458', 'PCBA-485313'), ('PCBA-2546', 'PCBA-485313'), ('PCBA-1030', 'PCBA-485364'), ('PCBA-504332', 'PCBA-485364'), ('PCBA-1460', 'PCBA-485364'), ('PCBA-1030', 'PCBA-504332'), ('PCBA-485364', 'PCBA-504332'), ('PCBA-1460', 'PCBA-504332'), ('PCBA-2551', 'PCBA-504333'), ('PCBA-1460', 'PCBA-504333'), ('PCBA-2546', 'PCBA-504333')]


In [ ]:
#Train pre-trained GNN on target task

results_df = pd.DataFrame(columns = ['task', 'source'] + dataset_metrics)
for source_task, target_task in source_target_pairs:
    print(f"{source_task}->{target_task}")

    train_loader, val_loader, test_loader, data_args = load_data(
        dataset=dataset_name,data_path=data_path,tasks=[target_task],
        device = device
    )
    model = load_model(
        device=device,
        source_model_path=source_model_path+f"{source_task}.pth"
    )
    trainer = Trainer(
        device=device,tasks=[target_task],
        data_args=data_args,model_path=model_path)
    _, task_results_dict = trainer.fit(model, train_loader, val_loader, test_loader)
    result = {'task': target_task, 'source': source_task}
    for metric in data_args['metrics']:
        if metric in dataset_metrics:
          result[metric] = task_results_dict[metric][target_task]
    results_df = results_df.append(result, ignore_index=True)
    result_path = model_path.replace('saved_models','results')
    makedir(result_path)
    results_df.to_csv(result_path+'fullresults.csv', float_format='%.3f',index=False)
    print(f"Results have been saved to {result_path+'fullresults.csv'}")

PCBA-485364->PCBA-1030
819 loaded!
100 loaded!
980 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/10000/PCBA-485364.pth
[0] training loss:0.6870503069138995
val roc_auc:0.6326530612244898
val auprc:0.6494529589230295
[20] training loss:0.5824093812820959
val roc_auc:0.6818727490996399
val auprc:0.6926969152027493
test roc_auc:0.617594501718213
test auprc:0.5744174724928834
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/PCBA/GCN/10000->1000/fullresults.csv
PCBA-504332->PCBA-1030


<ipython-input-26-67f066980aa5>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


819 loaded!
100 loaded!
980 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/10000/PCBA-504332.pth
[0] training loss:0.6745180908371421
val roc_auc:0.6602641056422569
val auprc:0.6638113710076428
[20] training loss:0.5632662568606582
val roc_auc:0.674669867947179
val auprc:0.7065161218532967
[40] training loss:0.5250611042275148
val roc_auc:0.6922769107643058
val auprc:0.7065565455891402
[60] training loss:0.5018046469080681
val roc_auc:0.6678671468587435
val auprc:0.686274290190892
test roc_auc:0.6297448713943559
test auprc:0.6132817131192729
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/PCBA/GCN/10000->1000/fullresults.csv
PCBA-1460->PCBA-1030


<ipython-input-26-67f066980aa5>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


819 loaded!
100 loaded!
980 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/10000/PCBA-1460.pth
[0] training loss:0.8115560885737924
val roc_auc:0.6654661864745898
val auprc:0.6701190274235618
[20] training loss:0.525273944232978
val roc_auc:0.651860744297719
val auprc:0.6307271632204399
test roc_auc:0.5966177236280329
test auprc:0.5691951851507879
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/PCBA/GCN/10000->1000/fullresults.csv
PCBA-485297->PCBA-1458


...

<ipython-input-26-67f066980aa5>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


770 loaded!
91 loaded!
954 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/10000/PCBA-1460.pth
[0] training loss:0.7342518276224533
val roc_auc:0.6917073170731708
val auprc:0.7448132220785497
[20] training loss:0.49414788372814655
val roc_auc:0.6819512195121952
val auprc:0.7273821939695994
test roc_auc:0.6758132583084226
test auprc:0.6730225347965444
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/PCBA/GCN/10000->1000/fullresults.csv
PCBA-2546->PCBA-504333


<ipython-input-26-67f066980aa5>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


770 loaded!
91 loaded!
954 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/10000/PCBA-2546.pth
[0] training loss:0.7093299155433973
val roc_auc:0.5756097560975609
val auprc:0.6346200023410774
[20] training loss:0.5676828864961863
val roc_auc:0.6785365853658537
val auprc:0.7199198599771313
test roc_auc:0.6663838579215756
test auprc:0.6617230803340634
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/PCBA/GCN/10000->1000/fullresults.csv


<ipython-input-26-67f066980aa5>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


In [ ]:
#Select best-performing model

import pandas as pd

results_path = path_to_src + f"/../results/{dataset_name}/GCN/10000->1000/"
results_df = pd.read_csv(results_path + 'fullresults.csv')
idx = results_df.groupby(['task'])[metric].transform(max) == results_df[metric]
results_df = results_df[idx]
results_df = results_df.drop_duplicates(['task']) #If two source tasks have the same score, just choose one randomly
results_df.to_csv(results_path + 'results.csv', index = False)